### Encontrar los *k* elementos más grandes

Cuando se tiene un conjunto de `n` elementos y se desea obtener únicamente los `k` elementos de mayor valor, ordenar la totalidad o utilizar algoritmos ingenuos (como seleccionar el máximo repetidamente) puede resultar ineficiente, especialmente cuando `n` es muy grande y `k` es relativamente pequeño.

#### Enfoque con heap

La idea es mantener un heap de tamaño máximo `k` que contenga los elementos de mayor valor vistos hasta el momento. Para ello se utiliza un **min-heap** (en lugar de un max-heap) de modo que el elemento en la raíz sea el menor de los *k* elementos almacenados. Así, para cada nuevo elemento del conjunto:

- **Si el heap aún no tiene `k` elementos:** se inserta el nuevo elemento.
- **Si ya tiene `k` elementos:** se compara el nuevo elemento con el que está en la raíz (el mínimo entre los *k* elementos actuales). Si el nuevo es mayor, se elimina la raíz (ya que no está entre los *k* elementos más grandes) y se inserta el nuevo elemento.

Este método permite procesar la lista en una única pasada, y como el heap tiene tamaño `k`, el costo total es de **O(n + klog(k))**, lo que representa una mejora sustancial cuando `k << n`.

#### Pseudocódigo

```
function topK(A, k) 
  heap ← DWayHeap()          // Crear un min-heap vacío
  for el in A do             
    if (heap.size == k and heap.peek() < el) then
      heap.top()             // Eliminar la raíz (el mínimo) del heap
    if (heap.size < k) then
      heap.insert(el)        // Insertar el nuevo elemento en el heap
  return heap                // Al final, el heap contiene los k elementos más grandes


**Código (implementación de DWayHeap y función topK)**

In [ ]:
from typing import Any, List, Optional, Tuple

class DWayHeap(object):
    def __init__(self, elements: List[Any] = [], priorities: List[float] = [], branching_factor: int = 2) -> None:
        """Constructor

        Args:
            elements: Los elementos para inicializar el heap.
            priorities: Las prioridades de los elementos anteriores. Deben tener la misma longitud que `elements`.
            branching_factor: El número (máximo) de hijos por nodo en el heap. Debe ser al menos 2.
        """
        if len(elements) != len(priorities):
            raise ValueError(f'La longitud de la lista de elementos ({len(elements)})'
                             f' debe coincidir con la longitud de la lista de prioridades ({len(priorities)}).')
        if branching_factor < 2:
            raise ValueError(f'El factor de ramificación ({branching_factor}) debe ser mayor que 1.')
        self._pairs: List[Tuple[float, Any]] = []
        self.D = branching_factor

        if len(elements) > 0:
            self._heapify(elements, priorities)

    def __sizeof__(self) -> int:
        """Tamaño del heap.

        Returns: El número de elementos en el heap.
        """
        return len(self)

    def __len__(self) -> int:
        """Tamaño del heap.

        Returns: El número de elementos en el heap.
        """
        return len(self._pairs)

    def _validate(self) -> bool:
        """Verifica que se cumplan las tres invariantes del heap:
        1. Cada nodo tiene como máximo `D` hijos.
        2. El árbol del heap es completo y alineado a la izquierda.
        3. Cada nodo contiene la mayor prioridad en el subárbol con raíz en ese nodo.

        Returns: True si se cumplen todas las invariantes del heap.
        """
        current_index = 0
        first_leaf = self.first_leaf_index()
        while current_index < first_leaf:
            current_priority: float = self._pairs[current_index][0]
            first_child = self._first_child_index(current_index)
            last_child_guard = min(first_child + self.D, len(self))
            for child_index in range(first_child, last_child_guard):
                if current_priority < self._pairs[child_index][0]:
                    return False
            current_index += 1
        return True

    def _push_down(self, index: int) -> None:
        """Empuja hacia abajo la raíz de un sub-heap para restablecer las invariantes del heap."""
        assert (0 <= index < len(self._pairs))
        input_pair = self._pairs[index]
        input_priority = input_pair[0]
        current_index = index
        first_leaf = self.first_leaf_index()
        while current_index < first_leaf:
            child_index = self._highest_priority_child_index(current_index)
            assert (child_index is not None)
            if self._pairs[child_index][0] > input_priority:
                self._pairs[current_index] = self._pairs[child_index]
                current_index = child_index
            else:
                break
        self._pairs[current_index] = input_pair

    def _bubble_up(self, index: int) -> None:
        """Eleva un elemento hacia la raíz para restablecer las invariantes del heap."""
        assert (0 <= index < len(self._pairs))
        input_pair = self._pairs[index]
        input_priority = input_pair[0]
        while index > 0:
            parent_index = self._parent_index(index)
            parent = self._pairs[parent_index]
            if input_priority > parent[0]:
                self._pairs[index] = parent
                index = parent_index
            else:
                break
        self._pairs[index] = input_pair

    def _first_child_index(self, index: int) -> int:
        """Calcula el índice del primer hijo de un nodo en el heap."""
        return index * self.D + 1

    def _parent_index(self, index: int) -> int:
        """Calcula el índice del padre de un nodo en el heap."""
        return (index - 1) // self.D

    def _highest_priority_child_index(self, index: int) -> Optional[int]:
        """Encuentra el hijo con mayor prioridad entre los hijos de un nodo."""
        first_index = self._first_child_index(index)
        size = len(self)
        last_index = min(first_index + self.D, size)

        if first_index >= size:
            return None

        highest_priority = -float('inf')
        index_result = first_index
        for i in range(first_index, last_index):
            if self._pairs[i][0] > highest_priority:
                highest_priority = self._pairs[i][0]
                index_result = i

        return index_result

    def first_leaf_index(self) -> int:
        return (len(self) - 2) // self.D + 1

    def _heapify(self, elements: List[Any], priorities: List[float]) -> None:
        """Inicializa el heap con una lista de elementos y sus prioridades."""
        assert (len(elements) == len(priorities))
        self._pairs = list(zip(priorities, elements))
        last_inner_node_index = self.first_leaf_index() - 1
        for index in range(last_inner_node_index, -1, -1):
            self._push_down(index)

    def is_empty(self) -> bool:
        """Verifica si el heap está vacío."""
        return len(self) == 0

    def top(self) -> Any:
        """Elimina y devuelve el elemento con mayor prioridad en el heap."""
        if self.is_empty():
            raise RuntimeError('Se llamó al método top en un heap vacío.')
        if len(self) == 1:
            element = self._pairs.pop()[1]
        else:
            element = self._pairs[0][1]
            self._pairs[0] = self._pairs.pop()
            self._push_down(0)
        return element

    def peek(self) -> Any:
        """Devuelve, sin eliminar, el elemento con mayor prioridad en el heap."""
        if self.is_empty():
            raise RuntimeError('Se llamó al método peek en un heap vacío.')
        return self._pairs[0][1]

    def insert(self, element: Any, priority: float) -> None:
        """Agrega un nuevo par (elemento, prioridad) al heap."""
        self._pairs.append((priority, element))
        self._bubble_up(len(self._pairs) - 1)


# Función ejemplo topK
# Dado que la implementación de DWayHeap es un max-heap, para simular un min-heap
# usamos el truco de insertar con prioridad negativa (es decir, la prioridad es -elemento)
# Esto es adecuado cuando los elementos son valores numéricos.

def topK(A: List[float], k: int) -> List[float]:
    """
    Retorna los k elementos más grandes de la lista A usando un heap de tamaño k.
    Se inserta cada elemento con prioridad negativa para simular un min-heap.

    Args:
        A: Lista de números.
        k: Número de elementos más grandes a encontrar.

    Returns:
        Una lista con los k elementos más grandes (sin orden específico).
    """
    heap = DWayHeap()  # Creamos un heap vacío
    for el in A:
        if len(heap) == k:
            # heap.peek() devuelve el mínimo de los k elementos (por la prioridad negativa)
            if heap.peek() < el:
                heap.top()  # Removemos el elemento mínimo
        if len(heap) < k:
            heap.insert(el, -el)  # Insertamos con prioridad negativa
    # Extraemos los elementos del heap
    return [pair[1] for pair in heap._pairs]

# Ejemplo de uso
datos = [2, 4, 1, 3, 7, 6, 18, 12]
k = 3
resultado = topK(datos, k)
print(f"Los {k} elementos más grandes son: {resultado}")


### Pruebas

In [ ]:
import unittest
import random

# Se asume que la clase DWayHeap y la función topK ya están definidas en el entorno.

BRANCHING_FACTORS_TO_TEST = [2, 3, 4, 5, 6]

class HeapTest(unittest.TestCase):
    def test_init(self):
        # Prueba de inicialización: se crea un heap vacío con factor de ramificación 2.
        heap = DWayHeap(branching_factor=2)
        self.assertEqual(0, len(heap))

        # Prueba con valores inválidos para el factor de ramificación (debe ser mayor que 1).
        for b in [1, 0, -1]:
            with self.assertRaises(ValueError) as context:
                DWayHeap(branching_factor=b)
            self.assertTrue(f'El factor de ramificación ({b}) debe ser mayor que 1.' in str(context.exception))

        # Prueba de inconsistencia entre la longitud de la lista de elementos y la lista de prioridades.
        with self.assertRaises(ValueError) as context:
            DWayHeap(priorities=[1.0])
        error_str = 'La longitud de la lista de elementos (0) debe coincidir con la longitud de la lista de prioridades (1).'
        self.assertTrue(error_str in str(context.exception))

        # Prueba de inicialización con elementos y prioridades válidos.
        heap = DWayHeap(elements=['A', 'B', 'C', 'D'], priorities=[0.1, -0.1, 1.0, -2.0], branching_factor=2)
        self.assertEqual(4, len(heap))
        self.assertTrue(heap._validate())

    def test_heapify(self):
        # Prueba de la función heapify para distintos factores de ramificación.
        for b in BRANCHING_FACTORS_TO_TEST:
            # Se genera un tamaño aleatorio para el heap.
            size = 4 + random.randint(0, 20)
            # Se generan elementos (letras) y prioridades aleatorias.
            elements = [chr(i) for i in range(ord('A'), ord('A') + size)]
            priorities = [random.random() for _ in range(size)]
            heap = DWayHeap(elements=elements, priorities=priorities, branching_factor=b)
            self.assertEqual(size, len(heap))
            self.assertTrue(heap._validate())

    def test_clear(self):
        # Prueba del comportamiento en heaps vacíos y tras insertar elementos.
        for b in BRANCHING_FACTORS_TO_TEST:
            heap = DWayHeap(branching_factor=b)
            # Se verifica que llamar a peek en un heap vacío arroja RuntimeError.
            with self.assertRaises(RuntimeError) as context:
                heap.peek()
            self.assertTrue('Se llamó al método peek en un heap vacío.' in str(context.exception))

            # Inserta el primer elemento con una prioridad muy baja.
            heap.insert('First', -1e14)
            self.assertEqual('First', heap.peek())

            # Inserta varios elementos con distintas prioridades.
            heap.insert("b", 0)
            heap.insert("c", 0.99)
            heap.insert("second", 1)
            heap.insert("a", -11)
            self.assertEqual('second', heap.peek())

    def test_insert_top(self):
        # Prueba de la inserción y extracción (top) de elementos en el heap.
        for b in BRANCHING_FACTORS_TO_TEST:
            heap = DWayHeap(branching_factor=b)
            # Verifica que llamar a peek en un heap vacío arroja RuntimeError.
            with self.assertRaises(RuntimeError) as context:
                heap.peek()
            self.assertTrue('Se llamó al método peek en un heap vacío.' in str(context.exception))

            # Inserta el primer elemento y lo extrae utilizando top.
            heap.insert('First', -1e14)
            self.assertEqual('First', heap.top())
            self.assertTrue(heap.is_empty())

            # Inserta varios elementos y verifica que se extrae el de mayor prioridad.
            heap.insert("b", 0)
            heap.insert("c", 0.99)
            heap.insert("second", 1)
            heap.insert("a", -11)
            self.assertEqual('second', heap.top())
            self.assertEqual(3, len(heap))

            # Inserta 10 elementos adicionales con prioridades aleatorias.
            for i in range(10):
                heap.insert(str(i), random.random())

            # Extrae elementos hasta que el heap esté vacío, verificando la estructura en cada extracción.
            while not heap.is_empty():
                self.assertTrue(heap._validate())
                heap.top()

    def test_topK(self):
        # Prueba de la función topK con una lista conocida.
        datos = [2, 4, 1, 3, 7, 6, 18, 12]
        k = 3
        resultado = topK(datos, k)
        # Los k elementos más grandes (sin orden específico) deben ser: 18, 12 y 7.
        expected = {18, 12, 7}
        self.assertEqual(expected, set(resultado))

        # Prueba adicional con elementos duplicados.
        datos = [5, 1, 5, 3, 9, 5]
        k = 2
        resultado = topK(datos, k)
        # Los dos mayores deberían ser 9 y 5.
        expected = {9, 5}
        self.assertEqual(expected, set(resultado))

suite = unittest.TestLoader().loadTestsFromTestCase(HeapTest)
unittest.TextTestRunner(verbosity=2).run(suite)

### Ejercicios

1.  Extiende la clase para incluir un método `update_priority(element, new_priority)` que permita cambiar la prioridad de un elemento ya existente en el heap y restablezca las invariantes del heap.  

*Retos:*  
- Localizar la posición del elemento de forma eficiente.  
- Decidir si hay que hacer _bubble up_ o _push down_ tras la actualización.  
- Diseñar casos de prueba que verifiquen que la invariante se mantiene tras la actualización.

2. Implementa un método `remove(element)` que permita eliminar cualquier elemento dado del heap sin violar las invariantes.  

*Retos:*  

- Encontrar el elemento (considera cómo manejar elementos duplicados si se permiten).  
- Reorganizar el heap tras la eliminación para restaurar la propiedad de heap.  
- Diseñar pruebas unitarias que verifiquen la eliminación correcta y la integridad del heap.

3. Ajusta la función `topK` y, de ser necesario, la estructura del heap para manejar adecuadamente elementos duplicados. Además, modifica `topK` para que retorne los `k` elementos más grandes ordenados de forma descendente.  

*Retos:*  

- Asegurar que el manejo de duplicados no afecte la complejidad o la invariante.  
- Realizar pruebas con conjuntos de datos que incluyan valores repetidos.

4. Realiza un análisis teórico de la complejidad en tiempo de las operaciones `insert`, `top` y (si se implementa) `update_priority` en función del factor de ramificación $D$. Luego, implementa pruebas empíricas (por ejemplo, con grandes volúmenes de datos) para comparar el rendimiento al variar $D$.  

*Retos:*  

- Formular hipótesis teóricas sobre cómo varía la complejidad.  
- Medir y graficar los tiempos de ejecución en distintos escenarios.

5. Utiliza la implementación del heap para crear una función `heap_sort` que ordene una lista de números. Realiza la ordenación tanto en orden ascendente como descendente.  

*Retos:*  

- Adaptar el heap (o la función `topK`) para extraer los elementos en el orden deseado.  
- Comparar el rendimiento de tu `heap_sort` con otros algoritmos de ordenación.

6. Modifica la implementación para que el heap pueda trabajar con elementos genéricos, utilizando una función de comparación (o una clave de ordenación) en lugar de depender de prioridades numéricas explícitas.  

*Retos:*  
   
- Diseñar una interfaz flexible para la comparación.  
- Adaptar todas las operaciones del heap para usar esta función de comparación.

7. Implementa un método adicional en la clase que imprima o genere una representación gráfica (por ejemplo, en forma de árbol) del heap.  

*Retos:*  

- Diseñar un formato de salida que muestre claramente la estructura y los niveles del heap.  
- Integrar la visualización en pruebas o en un entorno interactivo.

8.  Diseña un conjunto de pruebas que realicen un gran número de operaciones aleatorias (inserciones, extracciones, actualizaciones, eliminaciones) sobre el heap para asegurarte de que las invariantes siempre se mantengan.  

*Retos:*  

- Generar secuencias de operaciones que puedan revelar casos límite.  
- Automatizar la verificación de las invariantes tras cada operación.

9. Investiga y propone una variante inmutable (o persistente) del heap, donde cada modificación retorne una nueva versión del heap sin alterar la original.  

*Retos:*  
   
- Diseñar una estructura de datos que permita compartir partes comunes sin comprometer la inmutabilidad.  
- Evaluar el impacto en el rendimiento y la complejidad.

10. Revisa la implementación y busca oportunidades para optimizar la gestión de memoria y el rendimiento, especialmente cuando se trabaja con volúmenes muy grandes de datos.  

*Retos:*  

- Identificar cuellos de botella en el código.  
- Probar distintas estrategias (por ejemplo, ajustar el factor de ramificación) y documentar el impacto en el rendimiento.

Aquí tienes algunos ejercicios adicionales que te ayudarán a profundizar en el tema y a poner en práctica los conceptos aprendidos:

11. Dado un arreglo de números, implementa una función `kSmallest` que encuentre los `k` elementos con el menor valor. En lugar de utilizar un min-heap, utiliza un **max-heap** para mantener los `k` elementos más pequeños vistos hasta el momento.  
**Pistas:**
- Inserta los elementos en el heap con su valor original, ya que en un max-heap la raíz contendrá el máximo de los `k` elementos pequeños.
- Si al insertar un nuevo elemento el heap ya tiene `k` elementos, compara el nuevo valor con la raíz. Si es menor, elimina la raíz e inserta el nuevo valor.
- Analiza la complejidad temporal y espacial del algoritmo.

12. Imagina que los datos llegan de forma continua (por ejemplo, de un stream). Implementa una función que mantenga los 10 elementos más grandes en memoria de manera dinámica.

**Pistas:**  

- Usa la misma estrategia de min-heap de tamaño 10.  
- Simula la llegada de datos con una lista de números y procesa cada nuevo elemento actualizando el heap según corresponda.  
- Considera qué sucede si el flujo es infinito o muy grande y cómo se puede adaptar el algoritmo a estas situaciones.

13. Modifica la clase `DWayHeap` para que pueda funcionar tanto como min-heap como max-heap. Agrega un parámetro de inicialización (por ejemplo, `tipo`) que permita seleccionar el comportamiento.  

**Pistas:**  

- Ajusta las comparaciones en los métodos `_bubble_up`, `_push_down` y `_highest_priority_child_index` según el tipo de heap deseado.
- Asegúrate de que los métodos `insert`, `top` y `peek` respeten la semántica del tipo de heap configurado.
  
14. Supón que tienes un arreglo de objetos (por ejemplo, diccionarios) en el que cada objeto tiene un campo numérico llamado `"prioridad"`. Implementa una función que, utilizando un heap, extraiga los *k* objetos con la mayor prioridad.  

**Pistas:**  
- Define la prioridad para cada objeto según el valor del campo `"prioridad"`.
- Puedes reutilizar la estrategia de min-heap (usando la prioridad negativa, por ejemplo) para almacenar solo los `k` objetos de mayor prioridad.
- Considera cómo generalizar la función `topK` para trabajar con estructuras de datos más complejas.

15. Realiza un experimento para comparar el rendimiento del algoritmo `topK` basado en heap con la estrategia de ordenar la lista completa y luego seleccionar los `k` elementos mayores.  

**Pistas:**  
- Genera arreglos de diferentes tamaños (por ejemplo, 10⁴, 10⁵, 10⁶ elementos) y utiliza distintos valores de `k` (por ejemplo, 10, 100, 1000).  
- Mide el tiempo de ejecución de cada método y analiza en qué escenarios resulta más eficiente el uso de heaps.
- Presenta tus resultados mediante gráficos o tablas y discute las diferencias encontradas.


In [ ]:
## Tus respuestas